## Импорт библиотек

In [4]:
%%time
import gzip
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
%matplotlib inline
from IPython.display import clear_output
from ipywidgets.widgets import interact, interact_manual
import os
from PIL import Image
from multiprocessing.pool import Pool
from functools import partial
import sentencepiece as spm
import json
import gensim
from gensim.test.utils import datapath
import logging
import time
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

CPU times: user 3.41 s, sys: 5.07 s, total: 8.48 s
Wall time: 838 ms


/data/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Вывод всех сгенерированных файлов с последовтельностями

In [5]:
!ls files/*seq*

files/seqs_city_name.gz		     files/seqs_date_sorting_val.gz
files/seqs_cpe_manufacturer_name.gz  files/seqs_price.gz
files/seqs_cpe_model_name.gz	     files/seqs_region_name.gz
files/seqs_cpe_model_os_type.gz      files/seqs_request_cnt.gz
files/seqs_cpe_type_cd.gz	     files/seqs_url_host.gz


## Пример считывания одного файла

In [5]:
XX = []
c = 0
with gzip.open('files/seqs_cpe_model_name.gz', 'rt', encoding='utf-8') as f:
    for line in tqdm(f):
        #if c%10==0:
        XX.append(line.strip().split(' '))
        c += 1
len(XX)

0it [00:00, ?it/s]

415317

## Выставляем тэги документа для каждого юзера

In [6]:
tags = [[str(i)] for i in range(len(XX))]

## Учим Doc2Vec на разных окнах и числе эмбедов

In [7]:
%%time
class DocCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __init__(self, fold=1):
        print('Start initializing %d'%fold)
        print('Finished initializing %d'%fold)

    def __iter__(self):
        for tag_ids, tokens in zip(tags, XX):
            # assume there's one document per line, tokens separated by whitespace
            yield gensim.models.doc2vec.TaggedDocument(tokens,
                                                       tag_ids)

def train_doc2vec(args):
    WSIZE, EMBSIZE = args
    model_path = 'word2vec_models/cpemodname_d2v_tokens_w=%d_emb=%d_thr=5'%(WSIZE, EMBSIZE)
    if os.path.exists(model_path):
        return
    documents = DocCorpus()
    doc2vec_model = gensim.models.doc2vec.Doc2Vec(documents,
                                                  workers=4,
                                                  window=WSIZE,
                                                  vector_size=EMBSIZE,
                                                  min_count=20,
                                                  epochs=10)
    os.makedirs(model_path, exist_ok=True)
    doc2vec_model.save('%s/model'%(model_path))

args = []
candidates = [3, 128], [5, 64], [7, 256]
for candidate in candidates:
    args.append([*candidate])

args = np.array(args)
args = [arguments for arguments in args[np.random.permutation(len(args))].copy()]

for arguments in tqdm(args):
    train_doc2vec(arguments)
    time.sleep(1)

  0%|          | 0/3 [00:00<?, ?it/s]

2023-03-04 16:14:44,781 : INFO : collecting all words and their counts
2023-03-04 16:14:44,782 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags


Start initializing 1
Finished initializing 1


2023-03-04 16:14:45,703 : INFO : PROGRESS: at example #10000, processed 7792919 words (8458121/s), 404 word types, 10000 tags
2023-03-04 16:14:46,632 : INFO : PROGRESS: at example #20000, processed 15646370 words (8465818/s), 453 word types, 20000 tags
2023-03-04 16:14:52,387 : INFO : PROGRESS: at example #30000, processed 23496756 words (1364243/s), 486 word types, 30000 tags
2023-03-04 16:14:53,358 : INFO : PROGRESS: at example #40000, processed 31484172 words (8231941/s), 509 word types, 40000 tags
2023-03-04 16:14:54,294 : INFO : PROGRESS: at example #50000, processed 39289387 words (8351158/s), 529 word types, 50000 tags
2023-03-04 16:14:55,227 : INFO : PROGRESS: at example #60000, processed 47052059 words (8332395/s), 542 word types, 60000 tags
2023-03-04 16:14:56,181 : INFO : PROGRESS: at example #70000, processed 54897738 words (8235048/s), 551 word types, 70000 tags
2023-03-04 16:14:57,134 : INFO : PROGRESS: at example #80000, processed 62672255 words (8158945/s), 563 word typ

2023-03-04 16:17:09,508 : INFO : EPOCH 1 - PROGRESS: at 19.91% examples, 1786167 words/s, in_qsize 7, out_qsize 0
2023-03-04 16:17:10,508 : INFO : EPOCH 1 - PROGRESS: at 21.02% examples, 1786699 words/s, in_qsize 7, out_qsize 0
2023-03-04 16:17:11,512 : INFO : EPOCH 1 - PROGRESS: at 22.13% examples, 1786864 words/s, in_qsize 7, out_qsize 0
2023-03-04 16:17:12,515 : INFO : EPOCH 1 - PROGRESS: at 23.29% examples, 1790265 words/s, in_qsize 7, out_qsize 0
2023-03-04 16:17:13,516 : INFO : EPOCH 1 - PROGRESS: at 24.40% examples, 1789795 words/s, in_qsize 7, out_qsize 0
2023-03-04 16:17:14,516 : INFO : EPOCH 1 - PROGRESS: at 25.59% examples, 1792069 words/s, in_qsize 7, out_qsize 0
2023-03-04 16:17:15,519 : INFO : EPOCH 1 - PROGRESS: at 26.77% examples, 1793040 words/s, in_qsize 7, out_qsize 0
2023-03-04 16:17:16,520 : INFO : EPOCH 1 - PROGRESS: at 27.94% examples, 1796001 words/s, in_qsize 7, out_qsize 0
2023-03-04 16:17:17,524 : INFO : EPOCH 1 - PROGRESS: at 29.06% examples, 1800064 words/s

...

2023-03-04 17:06:30,259 : INFO : EPOCH 8 - PROGRESS: at 62.16% examples, 1786685 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:06:31,262 : INFO : EPOCH 8 - PROGRESS: at 63.17% examples, 1784863 words/s, in_qsize 8, out_qsize 0
2023-03-04 17:06:32,264 : INFO : EPOCH 8 - PROGRESS: at 64.23% examples, 1782968 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:06:33,266 : INFO : EPOCH 8 - PROGRESS: at 65.27% examples, 1781023 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:06:34,266 : INFO : EPOCH 8 - PROGRESS: at 66.30% examples, 1779407 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:06:35,268 : INFO : EPOCH 8 - PROGRESS: at 67.34% examples, 1777150 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:06:36,272 : INFO : EPOCH 8 - PROGRESS: at 68.39% examples, 1775714 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:06:37,273 : INFO : EPOCH 8 - PROGRESS: at 69.45% examples, 1773258 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:06:38,277 : INFO : EPOCH 8 - PROGRESS: at 70.47% examples, 1771257 words/s

2023-03-04 17:07:38,755 : INFO : EPOCH 9 - PROGRESS: at 33.44% examples, 1682480 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:07:39,756 : INFO : EPOCH 9 - PROGRESS: at 34.48% examples, 1682613 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:07:40,758 : INFO : EPOCH 9 - PROGRESS: at 35.55% examples, 1683153 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:07:41,760 : INFO : EPOCH 9 - PROGRESS: at 36.61% examples, 1683424 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:07:42,764 : INFO : EPOCH 9 - PROGRESS: at 37.63% examples, 1682052 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:07:43,765 : INFO : EPOCH 9 - PROGRESS: at 38.66% examples, 1680680 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:07:44,772 : INFO : EPOCH 9 - PROGRESS: at 39.73% examples, 1679278 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:07:45,773 : INFO : EPOCH 9 - PROGRESS: at 40.77% examples, 1679703 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:07:46,773 : INFO : EPOCH 9 - PROGRESS: at 41.82% examples, 1679392 words/s

2023-03-04 17:08:47,106 : INFO : EPOCH 10 - PROGRESS: at 4.01% examples, 1631530 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:08:48,106 : INFO : EPOCH 10 - PROGRESS: at 5.04% examples, 1632874 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:08:49,108 : INFO : EPOCH 10 - PROGRESS: at 6.04% examples, 1629428 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:08:50,108 : INFO : EPOCH 10 - PROGRESS: at 7.08% examples, 1638314 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:08:51,109 : INFO : EPOCH 10 - PROGRESS: at 8.11% examples, 1642983 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:08:52,111 : INFO : EPOCH 10 - PROGRESS: at 9.12% examples, 1651124 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:08:53,117 : INFO : EPOCH 10 - PROGRESS: at 10.19% examples, 1648935 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:08:54,119 : INFO : EPOCH 10 - PROGRESS: at 11.23% examples, 1656108 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:08:55,119 : INFO : EPOCH 10 - PROGRESS: at 12.25% examples, 1653300 word

2023-03-04 17:09:59,288 : INFO : EPOCH 10 - PROGRESS: at 78.45% examples, 1657733 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:10:00,290 : INFO : EPOCH 10 - PROGRESS: at 79.48% examples, 1657129 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:10:01,296 : INFO : EPOCH 10 - PROGRESS: at 80.52% examples, 1657073 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:10:02,297 : INFO : EPOCH 10 - PROGRESS: at 81.55% examples, 1657261 words/s, in_qsize 6, out_qsize 1
2023-03-04 17:10:03,298 : INFO : EPOCH 10 - PROGRESS: at 82.58% examples, 1657726 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:10:04,298 : INFO : EPOCH 10 - PROGRESS: at 83.61% examples, 1658369 words/s, in_qsize 6, out_qsize 1
2023-03-04 17:10:05,301 : INFO : EPOCH 10 - PROGRESS: at 84.65% examples, 1658454 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:10:06,305 : INFO : EPOCH 10 - PROGRESS: at 85.74% examples, 1658576 words/s, in_qsize 7, out_qsize 0
2023-03-04 17:10:07,306 : INFO : EPOCH 10 - PROGRESS: at 86.79% examples, 165893

CPU times: user 2h 15min 58s, sys: 6min 47s, total: 2h 22min 45s
Wall time: 55min 37s


## Повторяем проделанное выше для url_host, date и region файлов

## Получившиеся модели

In [11]:
!ls word2vec_models/*128*

word2vec_models/cpemodname_d2v_tokens_w=3_emb=128_thr=5:
model  model.docvecs.vectors_docs.npy

word2vec_models/d2v_tokens_w=3_emb=128_thr=5:
model  model.docvecs.vectors_docs.npy

word2vec_models/date_d2v_tokens_w=3_emb=128_thr=5:
model  model.docvecs.vectors_docs.npy

word2vec_models/region_d2v_tokens_w=3_emb=128_thr=5:
model  model.docvecs.vectors_docs.npy


## Загружаем их и конкатенируем их эмбеддинги документов в один эмбеддинг Doc2Vec

In [5]:
feats = []
pbar = tqdm()
for doc2vec in ['d2v_tokens_w=3_emb=128_thr=5',
                'date_d2v_tokens_w=3_emb=128_thr=5',
                'region_d2v_tokens_w=3_emb=128_thr=5',
                'cpemodname_d2v_tokens_w=3_emb=128_thr=5']:
    doc2vec = gensim.models.doc2vec.Doc2Vec.load('word2vec_models/%s/model'%doc2vec)
    feats.append(doc2vec.docvecs.vectors_docs.copy())
    pbar.update(1)
feats = np.concatenate(feats, axis=1)
with gzip.open('doc2vec_feats_128x4.pickle.gz', 'wb') as f:
    pickle.dump(feats, f, protocol=-1)

0it [00:00, ?it/s]

2023-03-04 20:07:15,633 : INFO : loading Doc2Vec object from word2vec_models/d2v_tokens_w=3_emb=128_thr=5/model
2023-03-04 20:07:16,749 : INFO : loading vocabulary recursively from word2vec_models/d2v_tokens_w=3_emb=128_thr=5/model.vocabulary.* with mmap=None
2023-03-04 20:07:16,750 : INFO : loading trainables recursively from word2vec_models/d2v_tokens_w=3_emb=128_thr=5/model.trainables.* with mmap=None
2023-03-04 20:07:16,750 : INFO : loading wv recursively from word2vec_models/d2v_tokens_w=3_emb=128_thr=5/model.wv.* with mmap=None
2023-03-04 20:07:16,751 : INFO : loading docvecs recursively from word2vec_models/d2v_tokens_w=3_emb=128_thr=5/model.docvecs.* with mmap=None
2023-03-04 20:07:16,751 : INFO : loading vectors_docs from word2vec_models/d2v_tokens_w=3_emb=128_thr=5/model.docvecs.vectors_docs.npy with mmap=None
2023-03-04 20:07:16,914 : INFO : loaded word2vec_models/d2v_tokens_w=3_emb=128_thr=5/model
2023-03-04 20:07:17,196 : INFO : loading Doc2Vec object from word2vec_models/

## Проверяем наличие файла

In [6]:
!ls doc2vec_feats_128x4.pickle.gz

doc2vec_feats_128x4.pickle.gz
